In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Longstaff - Schwartz Metódus

### Legelőször nézzük meg a példa feladatot a "Valuing American Options by Simulation: A Simple Least-Squares Approach" cikkből

Először is deklaráltam a cikkben leírt paramétereket:

In [2]:
N, r, K, T = 4, 0.06, 1.1, 3 

dt = T/(N-1)
df = np.exp(-r * dt)

In [3]:
S = np.array([[1.00, 1.09, 1.08, 1.34],
            [1.00, 1.16, 1.26, 1.54],
            [1.00, 1.22, 1.07, 1.03],
            [1.00, 0.93, 0.97, 0.92],
            [1.00, 1.11, 1.56, 1.52],
            [1.00, 0.76, 0.77, 0.90],
            [1.00, 0.92, 0.84, 1.01],
            [1.00, 0.88, 1.22, 1.34]])

print('Árfolyam trajektoriák:\n', S)

Árfolyam trajektoriák:
 [[1.   1.09 1.08 1.34]
 [1.   1.16 1.26 1.54]
 [1.   1.22 1.07 1.03]
 [1.   0.93 0.97 0.92]
 [1.   1.11 1.56 1.52]
 [1.   0.76 0.77 0.9 ]
 [1.   0.92 0.84 1.01]
 [1.   0.88 1.22 1.34]]


In [4]:
H = np.maximum(K - S, 0)
V = np.zeros_like(H)
V[:, -1] = H[:, -1]
print("Cash-flow mátrix a 3. időpillanatban:\n", V)

Cash-flow mátrix a 3. időpillanatban:
 [[0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.07]
 [0.   0.   0.   0.18]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.2 ]
 [0.   0.   0.   0.09]
 [0.   0.   0.   0.  ]]


### LS metódus az árazásra:

In [5]:
for t in range(N-2, 0, -1):
    good_paths = H[:,t] > 0
    rg = np.polyfit(S[good_paths, t], V[good_paths, t+1] * df, 2)
    C = np.polyval( rg, S[good_paths,t] )
    print('\n\t\t\tRegresszió '+str(t)+' időpillanatban:')
    print('X: ', S[good_paths, t])
    print('Y: ', V[good_paths, t+1] * df)
    print('\n\t\t\tPolinomiális regresszió:')
    print('E[ Y | X ] = ', round(rg[-1], 3), ' + ', round(rg[-2], 3),'* X', ' + ', round(rg[-3], 3), '* X^2' )
    print('\n\tFolytatási értékek '+str(t)+'-ben :\n', C)
    print('\tLehívási értékek '+str(t)+'-ben :\n', H[H[:, t] > 0][:, t])
    
    exercise = np.zeros( len(good_paths), dtype=bool)
    exercise[good_paths] = H[good_paths,t] > C
    
    V[exercise,t] = H[exercise,t]
    V[exercise,t+1:] = 0                                 
    discount_path = (V[:,t] == 0)
    V[discount_path,t] = V[discount_path,t+1] * df
    
V0 = np.mean(V[:,1]) * df


			Regresszió 2 időpillanatban:
X:  [1.08 1.07 0.97 0.77 0.84]
Y:  [0.         0.06592352 0.16951762 0.18835291 0.08475881]

			Polinomiális regresszió:
E[ Y | X ] =  -1.07  +  2.983 * X  +  -1.814 * X^2

	Folytatási értékek 2-ben :
 [0.03674056 0.04589834 0.11752682 0.15196921 0.15641792]
	Lehívási értékek 2-ben :
 [0.02 0.03 0.13 0.33 0.26]

			Regresszió 1 időpillanatban:
X:  [1.09 0.93 0.76 0.92 0.88]
Y:  [0.         0.12242939 0.3107823  0.24485878 0.        ]

			Polinomiális regresszió:
E[ Y | X ] =  2.038  +  -3.335 * X  +  1.356 * X^2

	Folytatási értékek 1-ben :
 [0.01348511 0.10874928 0.28606468 0.11700927 0.15276213]
	Lehívási értékek 1-ben :
 [0.01 0.17 0.34 0.18 0.22]


### Implementáltam a az algoritmust, és szimulációval áraztam be különböző put opciókat. Emellett megvizsgáltam, hogy ez az ár mennyivel tér el a binomiális fával kapott értékektől.

In [6]:
from binomialtree import BinomialTree
from longstaff_schwartz import Longstaff_Schwartz

A cikkben megtalálható "Table 1" paraméterei vannak, megadva, hogy lássuk tényleg úgy sikerült-e implementálni, ahogy a cikkben leírtak vannak. Emellett a binomiális fa segítségével is beáraztam az amerikai put opciók értékeit, ugyanazokkal a paraméterekkel:

In [7]:
S0V = np.array([])
sigV = np.array([])
TV = np.array([])
LSMV = np.array([])
LSMV_2 = np.array([])
putV = np.array([])

print(' \tLSM results:')
for S0 in (36., 38., 40., 42., 44.):
    for sig in (0.2, 0.4):
        for T in (1.0, 2.0):
            S0V = np.append(S0V, S0)
            sigV = np.append(sigV, sig)
            TV = np.append(TV, T)
            putValue, _, _ = Longstaff_Schwartz(S0, 40, 0.06, sig, T, N = int(T*50)).putprice()
            putValue2, _, _ = Longstaff_Schwartz(S0, 40, 0.06, sig, T, N = int(T*50)).putprice()
            LSMV = np.append(LSMV, putValue)
            LSMV_2 = np.append(LSMV_2, putValue2)
            print("S0 %4.1f | vol %4.2f | T %2.1f | Option Value %8.3f" % (S0, sig, T, putValue))
            
print('\n \tBinomial Model results:')
for S0 in (36., 38., 40., 42., 44.):
    for sig in (0.2, 0.4):
        for T in (1.0, 2.0):
            model = BinomialTree(S0, 0.06, sig, int(T), int(T*1000))
            putValue = model.putprice(40)[0,0]
            putV = np.append(putV, putValue)
            print("S0 %4.1f | vol %4.2f | T %2.1f | Option Value %8.3f" % (S0, sig, T, putValue))

 	LSM results:
S0 36.0 | vol 0.20 | T 1.0 | Option Value    4.429
S0 36.0 | vol 0.20 | T 2.0 | Option Value    4.850
S0 36.0 | vol 0.40 | T 1.0 | Option Value    7.067
S0 36.0 | vol 0.40 | T 2.0 | Option Value    8.444
S0 38.0 | vol 0.20 | T 1.0 | Option Value    3.243
S0 38.0 | vol 0.20 | T 2.0 | Option Value    3.708
S0 38.0 | vol 0.40 | T 1.0 | Option Value    6.121
S0 38.0 | vol 0.40 | T 2.0 | Option Value    7.668
S0 40.0 | vol 0.20 | T 1.0 | Option Value    2.310
S0 40.0 | vol 0.20 | T 2.0 | Option Value    2.848
S0 40.0 | vol 0.40 | T 1.0 | Option Value    5.365
S0 40.0 | vol 0.40 | T 2.0 | Option Value    6.917
S0 42.0 | vol 0.20 | T 1.0 | Option Value    1.616
S0 42.0 | vol 0.20 | T 2.0 | Option Value    2.245
S0 42.0 | vol 0.40 | T 1.0 | Option Value    4.557
S0 42.0 | vol 0.40 | T 2.0 | Option Value    6.229
S0 44.0 | vol 0.20 | T 1.0 | Option Value    1.111
S0 44.0 | vol 0.20 | T 2.0 | Option Value    1.691
S0 44.0 | vol 0.40 | T 1.0 | Option Value    3.930
S0 44.0 | vol 0.

Az eredményeket egy táblázatba foglaltam a jobb áttekinthetőség miatt:

In [8]:
diff = putV - LSMV
diff2 = putV - LSMV_2
df = pd.DataFrame(data= {'S': S0V, 'Sigma': sigV, 'T': TV, 'Binom_put': putV, 'LSM Put': LSMV, 'LSM Put 100': LSMV_2, 'Különbség_1': diff, 'Különbség_2': diff2})
display(df)

,S,Sigma,T,Binom_put,LSM_put_1,LSM_put_2,Különbség_1,Különbség_2
0,36.0,0.2,1.0,4.486837,4.429028,4.465968,0.057809,0.020870
1,36.0,0.2,2.0,4.848330,4.849704,4.801501,-0.001374,0.046829
2,36.0,0.4,1.0,7.109383,7.066757,7.084327,0.042626,0.025056
3,36.0,0.4,2.0,8.514380,8.444057,8.489258,0.070322,0.025122
4,38.0,0.2,1.0,3.257069,3.242850,3.221418,0.014219,0.035651
5,38.0,0.2,2.0,3.751247,3.707945,3.724318,0.043302,0.026929
6,38.0,0.4,1.0,6.154111,6.120795,6.154524,0.033317,-0.000413
7,38.0,0.4,2.0,7.674588,7.667758,7.597748,0.006830,0.076840
8,40.0,0.2,1.0,2.319278,2.310367,2.274098,0.008911,0.045180
9,40.0,0.2,2.0,2.889738,2.847772,2.875774,0.041966,0.013964


A LSM tendenciózusan alábecsli az opció árát a suboptimális lehívási korlát miatt. Azonban túl is tudja becsülni, annak köszönhetően, hogy ugyanazt a trajektoriát használja a döntéshozatalhoz és az értékeléshez.

In [11]:
import time
LSM_df = pd.DataFrame()
for i in range(100):
    start = time.time()
    LSM_arr = np.array([])
    for S0 in (36., 38., 40., 42., 44.):
        for sig in (0.2, 0.4):
            for T in (1.0, 2.0):
                putValue, _, _ = Longstaff_Schwartz(S0, 40, 0.06, sig, T, N = int(T*50)).putprice()
                LSM_arr = np.append(LSM_arr, putValue)
    print(i, round(time.time() - start, 4), 'sec')
    LSM_df[str(i)] = LSM_arr
    
LSM_des = LSM_df.T.describe()

0 12.3778 sec
1 12.1647 sec
2 12.5518 sec
3 12.4468 sec
4 12.3078 sec
5 12.1988 sec
6 12.2918 sec
7 12.1698 sec
8 12.1827 sec
9 12.1757 sec
10 11.9637 sec
11 12.3718 sec
12 12.0737 sec
13 11.9777 sec
14 11.7607 sec
15 12.1027 sec
16 12.0057 sec
17 11.8467 sec
18 11.8867 sec
19 12.2718 sec
20 12.0627 sec
21 11.9077 sec
22 12.0227 sec
23 11.9857 sec
24 12.0287 sec
25 11.9367 sec
26 12.1868 sec
27 12.1327 sec
28 11.8697 sec
29 12.1262 sec
30 12.0847 sec
31 11.9447 sec
32 12.2368 sec
33 11.9427 sec
34 11.9277 sec
35 12.0737 sec
36 11.8367 sec
37 11.9907 sec
38 11.8767 sec
39 11.8907 sec
40 11.7457 sec
41 11.8127 sec
42 11.7897 sec
43 11.7336 sec
44 11.9677 sec
45 11.8667 sec
46 11.6636 sec
47 11.8147 sec
48 11.8297 sec
49 11.8327 sec
50 12.5083 sec
51 11.8517 sec
52 11.9997 sec
53 11.9697 sec
54 11.9167 sec
55 11.9177 sec
56 12.0337 sec
57 12.1637 sec
58 11.8437 sec
59 11.8737 sec
60 12.5663 sec
61 11.9287 sec
62 12.0157 sec
63 11.8547 sec
64 11.7887 sec
65 11.8737 sec
66 11.8087 sec
67 11

In [23]:
diffi = putV - LSM_des.T['mean']
pd.DataFrame(data = {'S': S0V[:20], 'Sigma': sigV[:20], 'T': TV[:20], 'Binomial Value': putV[:20], 'LSM Value': LSM_des.T['mean'], 'LSM Std': LSM_des.T['std'], 'Differencia': diffi})

,S,Sigma,T,Binomial Value,LSM Value,LSM Std,Differencia
0,36.0,0.2,1.0,4.486837,4.467839,0.013466,0.018998
1,36.0,0.2,2.0,4.848330,4.823244,0.014603,0.025086
2,36.0,0.4,1.0,7.109383,7.087123,0.025703,0.022260
3,36.0,0.4,2.0,8.514380,8.495681,0.031976,0.018699
4,38.0,0.2,1.0,3.257069,3.237334,0.013367,0.019736
5,38.0,0.2,2.0,3.751247,3.729262,0.016479,0.021985
6,38.0,0.4,1.0,6.154111,6.139225,0.028831,0.014886
7,38.0,0.4,2.0,7.674588,7.654121,0.032487,0.020467
8,40.0,0.2,1.0,2.319278,2.305762,0.013575,0.013516
9,40.0,0.2,2.0,2.889738,2.869840,0.015449,0.019898


100-szor szimuláltam le az LSM árazást, annak a leíró statisztikája is megtalálható a kódban. Ami szemet szúrhat nekünk, hogy a 100 szimuláció átlagát nézve, mindenhol kivétel nélkül alulbecsüli a modellünk a binomiális fából kapott értéket